Import public packages and functions

In [1]:
import os
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as stat
import json

In [2]:
def get_repo_path_in_notebook():
    """
    Finds path of repo from Notebook.
    Start running this once to correctly find
    other modules/functions
    """
    path = os.getcwd()
    repo_name = 'ultraleap_analysis'

    while path[-len(repo_name):] != 'ultraleap_analysis':

        path = os.path.dirname(path)

    return path

In [3]:
repo_path = get_repo_path_in_notebook()
code_path = os.path.join(repo_path, 'code')
os.chdir(code_path)

Import own functions

In [4]:
import import_data.import_and_convert_data as import_dat
import import_data.find_paths as find_paths
import movement_calc.helpfunctions as hp
import sig_processing.calc_features as feat_ext

### Feature Extraction

Load dist-time .csv files and calculate features

In [5]:
folder = 'patientdata'
subs = find_paths.find_available_subs(folder)
ls_task = ['ft','oc','ps']
ls_cond = ['m0s0', 'm1s0', 'm1s1', 'm0s1', 'm1', 'm0']
ls_cam = ['vr', 'dt', 'st']
ls_block = ['b1', 'b2', 'b3']
ls_side = ['right', 'left']

In [10]:
importlib.reload(import_dat)
importlib.reload(find_paths)
importlib.reload(feat_ext)
importlib.reload(hp)

for sub in subs:
    for task in ls_task:
        for side in ls_side:
            for cond in ls_cond:
                for cam in ls_cam:
                    for block in ls_block:
                        try:                                            
                            # Load blocks from patients' blocks dir
                            dist = pd.read_csv(
                                os.path.join(
                                repo_path, 'data', sub, task, cond, 'distances', 
                                f'{block}_{sub}_{cond}_{cam}_{task}_{side}.csv'), 
                                index_col=0
                                )

                            get_feat = feat_ext.calculate_feat(dist, block, sub, cond, cam, task, side, 
                                                                to_plot = True, to_save_plot = True, to_save = True)

                        except FileNotFoundError:
                            print('No files available for combination: '
                                f'{block, sub, cond, cam, task, side}')

No files available for combination: ('b1', 'ul001', 'm0s0', 'vr', 'ft', 'right')
No files available for combination: ('b2', 'ul001', 'm0s0', 'vr', 'ft', 'right')
No files available for combination: ('b3', 'ul001', 'm0s0', 'vr', 'ft', 'right')
No files available for combination: ('b1', 'ul001', 'm0s0', 'dt', 'ft', 'right')
No files available for combination: ('b2', 'ul001', 'm0s0', 'dt', 'ft', 'right')
No files available for combination: ('b3', 'ul001', 'm0s0', 'dt', 'ft', 'right')
No files available for combination: ('b1', 'ul001', 'm0s0', 'st', 'ft', 'right')
No files available for combination: ('b2', 'ul001', 'm0s0', 'st', 'ft', 'right')
No files available for combination: ('b3', 'ul001', 'm0s0', 'st', 'ft', 'right')
No files available for combination: ('b1', 'ul001', 'm1s0', 'vr', 'ft', 'right')
No files available for combination: ('b2', 'ul001', 'm1s0', 'vr', 'ft', 'right')
No files available for combination: ('b3', 'ul001', 'm1s0', 'vr', 'ft', 'right')
No files available for combi

Create features dataframe 

In [68]:
# Define task
task = 'ft'

In [69]:
subs = find_paths.find_available_subs(folder)
list_of_feats = []
files = []

for sub in subs:
    for side in ls_side:
        for cond in ls_cond:
            for cam in ls_cam:
                for block in ls_block:

                    file = os.path.join(repo_path,'features', sub, task, cond, 'features_block',f'{block}_{sub}_{cond}_{cam}_{task}_{side}.json')
                    if not os.path.exists(file):
                        print('No files available for combination: '
                            f'{block, sub, cond, cam, task, side}')
                        continue  # skips non-existing files
                    with open(file) as f:
                        load_json = json.load(f)
                        feat_names = list(load_json.keys())
                        if not any(load_json.values()):
                            continue  # skips empty files
                        fts = np.array(list(load_json.values())).ravel()
                        list_of_feats.append(fts)
                        files.append(f'{block}_{sub}_{cond}_{cam}_{task}_{side}.json')

all_feats = np.array(list_of_feats)
df_fts = pd.DataFrame(data=all_feats, columns=feat_names)
df_fts.insert(loc=0, column='file', value=files)


No files available for combination: ('b1', 'ul001', 'm0s0', 'vr', 'ft', 'right')
No files available for combination: ('b2', 'ul001', 'm0s0', 'vr', 'ft', 'right')
No files available for combination: ('b3', 'ul001', 'm0s0', 'vr', 'ft', 'right')
No files available for combination: ('b1', 'ul001', 'm0s0', 'dt', 'ft', 'right')
No files available for combination: ('b2', 'ul001', 'm0s0', 'dt', 'ft', 'right')
No files available for combination: ('b3', 'ul001', 'm0s0', 'dt', 'ft', 'right')
No files available for combination: ('b1', 'ul001', 'm0s0', 'st', 'ft', 'right')
No files available for combination: ('b2', 'ul001', 'm0s0', 'st', 'ft', 'right')
No files available for combination: ('b3', 'ul001', 'm0s0', 'st', 'ft', 'right')
No files available for combination: ('b1', 'ul001', 'm1s0', 'vr', 'ft', 'right')
No files available for combination: ('b2', 'ul001', 'm1s0', 'vr', 'ft', 'right')
No files available for combination: ('b3', 'ul001', 'm1s0', 'vr', 'ft', 'right')
No files available for combi

In [70]:
df_fts

,file,num_events,mean_max_dist,sd_max_dist,coef_var_max_dist,slope_max_dist,mean_max_vel,sd_max_vel,coef_var_max_vel,slope_max_vel,...,coef_var_mean_vel,slope_mean_vel,mean_tap_dur,sd_tap_dur,coef_var_tap_dur,slope_tap_dur,mean_rms,sd_rms,slope_rms,sum_rms
0,b1_ul001_m1_vr_ft_right.json,31.0,0.134933,0.004083,0.030256,0.000072,2.559501,0.425132,0.166100,0.017827,...,0.094988,0.000828,0.360868,0.031789,0.088091,-0.000501,0.077272,0.004883,-0.000028,2.318146
1,b2_ul001_m1_vr_ft_right.json,26.0,0.139824,0.006147,0.043961,-0.000597,2.756138,0.472769,0.171533,-0.032398,...,0.102546,-0.007141,0.418023,0.146259,0.349883,0.002214,0.070272,0.007038,-0.000641,1.756795
2,b1_ul001_m1_dt_ft_right.json,24.0,0.100558,0.010255,0.101979,-0.000742,2.345756,0.899337,0.383389,-0.052527,...,0.174882,0.003284,0.427845,0.082475,0.192769,-0.001855,0.059499,0.004799,-0.000181,1.368483
3,b2_ul001_m1_dt_ft_right.json,20.0,0.086324,0.019164,0.222004,-0.001298,1.531217,0.322006,0.210294,-0.007304,...,0.239461,-0.010281,0.487451,0.093077,0.190946,-0.000302,0.050991,0.006480,-0.000532,0.968823
4,b3_ul001_m1_dt_ft_right.json,26.0,0.086305,0.016929,0.196150,-0.001409,1.894720,0.857292,0.452464,-0.074071,...,0.230737,-0.006922,0.437970,0.093196,0.212792,0.007941,0.050254,0.007442,-0.000370,1.256361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,b1_ul019_m0s1_dt_ft_left.json,5.0,0.042018,0.006991,0.166394,-0.005951,0.208265,0.075892,0.364401,-0.056226,...,0.435531,-0.018179,2.468707,1.100433,0.445753,0.970097,0.030241,0.002985,-0.002282,0.120964
195,b1_ul020_m1s1_dt_ft_right.json,19.0,0.081074,0.011147,0.137487,-0.000044,1.239993,0.449993,0.362899,0.003992,...,0.300623,0.000873,0.606661,0.176425,0.290813,-0.011463,0.042700,0.008176,-0.000243,0.768608
196,b1_ul020_m1s1_dt_ft_left.json,10.0,0.050135,0.017826,0.355548,-0.001690,0.822843,0.330159,0.401241,-0.022001,...,0.424440,-0.006008,0.871195,0.365085,0.419063,0.042304,0.016592,0.005847,-0.000681,0.149325
197,b2_ul020_m1s1_dt_ft_left.json,17.0,0.057558,0.010248,0.178050,-0.000462,0.895016,0.263797,0.294740,-0.007001,...,0.305599,-0.004356,0.689338,0.127394,0.184806,0.008581,0.025389,0.005153,-0.000159,0.406228


Save features dataframe in csv file

In [71]:
df_fts_path = os.path.join(repo_path,
                           'EDA',
                            task
                            ) 
if not os.path.exists(df_fts_path):
    os.makedirs(df_fts_path)
df_fts.to_csv(os.path.join(df_fts_path, 'features_new.csv'))